# Análisis Comparativo de Modelos Ensemble para Predicción de Consumo Energético
 
Este notebook implementa y compara diferentes algoritmos de ensemble learning para la predicción del consumo energético, incluyendo Random Forest, Gradient Boosting y XGBoost. El objetivo es identificar el mejor modelo y método de optimización de hiperparámetros.
 
## Estructura del Análisis
1. Carga y Preparación de Datos: Dataset preprocesado con adición de ruido realista
2. Modelos Base: Implementación de tres algoritmos ensemble principales
3. Optimización de Hiperparámetros: Comparación de GridSearch, RandomSearch y BayesianSearch
4. Evaluación Comparativa: Análisis de rendimiento entre todos los enfoques
5. Visualizaciones: Gráficos diagnósticos y comparativos
6. Persistencia: Guardado del mejor modelo para producción
 
### Configuración del Entorno
 
Librerías clave para ensemble learning:
- sklearn.ensemble: Random Forest y Gradient Boosting clásicos
- xgboost: Implementación optimizada de Gradient Boosting
- skopt: Optimización bayesiana de hiperparámetros
 
¿Por qué modelos ensemble?
Los métodos ensemble combinan múltiples modelos para obtener mejor rendimiento que cualquier modelo individual:
- Reducción de varianza: Promedio de múltiples predicciones reduce ruido
- Reducción de sesgo: Diferentes modelos capturan diferentes patrones
- Mayor robustez: Menos sensibles a outliers y ruido en datos


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
import warnings
from skopt import BayesSearchCV
from skopt.space import Real, Integer

warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')


### Carga de Datasets Preprocesados
 
Utilizamos los mismos datasets que en el análisis SVR para asegurar comparabilidad directa entre diferentes enfoques de modelado. Los datos ya han pasado por:
- Codificación de variables categóricas (one-hot encoding)
- Limpieza y tratamiento de valores faltantes
- Estructuración consistente entre train/test

In [4]:
# 1. CARGAR Y PREPARAR LOS DATOS
train_df = pd.read_csv('../data/processed/energy_data_processed.csv')
test_df = pd.read_csv('../data/processed/energy_data_processed_test.csv')

### Separación de Variables y Adición de Variabilidad Realista
 
Separación estándar:
- X_train, X_test: Matrices de características para entrenamiento y prueba
- y_train, y_test: Vectores de variable objetivo
 
Adición de ruido gaussiano:
Se mantiene la misma estrategia del notebook SVR para consistencia:
- noise_factor = 0.1: 10% del rango total de la variable objetivo
- Distribución normal: Ruido más realista que ruido uniforme
- Mismo seed: Garantiza reproducibilidad entre experimentos
 
Justificación del ruido:
- Datos perfectos son irreales: En la práctica siempre hay variabilidad no explicada
- Evita overfitting artificial: Modelos demasiado perfectos no generalizan bien
- Simula incertidumbre: Refleja errores de medición y factores no observados
 
Monitoreo de la transformación:
- Verificamos que el rango y varianza del ruido sean apropiados
- Mantenemos las propiedades estadísticas básicas de los datos originales


In [5]:
# 2. SEPARAR CARACTERÍSTICAS Y VARIABLE OBJETIVO
y_train = train_df['Energy Consumption']
X_train = train_df.drop('Energy Consumption', axis=1)
y_test = test_df['Energy Consumption']
X_test = test_df.drop('Energy Consumption', axis=1)

# Verificar tamaños
print(f'Forma del dataset de entrenamiento: {X_train.shape}')
print(f'Forma del dataset de prueba: {X_test.shape}')
print(f'Variable objetivo: {y_train.name}')

# Añadir ruido aleatorio para simular datos más realistas
np.random.seed(42)  # Para reproducibilidad
noise_factor = 0.1  # 10% del rango de y_train
y_train_range = y_train.max() - y_train.min()
y_train_noisy = y_train + np.random.normal(0, noise_factor * y_train_range, size=y_train.shape)
y_test_noisy = y_test + np.random.normal(0, noise_factor * y_train_range, size=y_test.shape)
print("\nVarianza de y_train con ruido:", y_train_noisy.var())
print("Rango de y_train con ruido: [{:.2f}, {:.2f}]".format(y_train_noisy.min(), y_train_noisy.max()))
print("Varianza de y_test con ruido:", y_test_noisy.var())
print("Rango de y_test con ruido: [{:.2f}, {:.2f}]".format(y_test_noisy.min(), y_test_noisy.max()))

Forma del dataset de entrenamiento: (1000, 9)
Forma del dataset de prueba: (100, 9)
Variable objetivo: Energy Consumption

Varianza de y_train con ruido: 1086917.99596434
Rango de y_train con ruido: [1442.89, 6860.07]
Varianza de y_test con ruido: 973255.7143404601
Rango de y_test con ruido: [1875.14, 6906.85]


### Escalado de Características para Modelos Ensemble
 
¿Es necesario escalar para árboles de decisión?
- Random Forest/Gradient Boosting: No requieren escalado (son invariantes a transformaciones monótonas)
- XGBoost: Tampoco requiere escalado estrictamente
 
¿Por qué escalamos entonces?
1. Consistencia experimental: Mantener las mismas condiciones que en SVR
2. Comparabilidad: Eliminamos el escalado como variable confundidora
3. Futuros experimentos: Si queremos probar modelos híbridos o ensemble con algoritmos sensibles al escalado
4. Regularización: Algunos parámetros de regularización pueden funcionar mejor con datos escalados
 
Proceso de escalado:
- fit_transform en entrenamiento: Calcula media y desviación estándar
- transform en prueba: Aplica la misma transformación sin recalcular estadísticas
- Prevención de data leakage: No usamos información de test para el escalado

In [6]:
# 3. ESCALADO DE CARACTERÍSTICAS (opcional para árboles, pero útil para consistencia)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Implementación de Modelos Ensemble Base
 
Configuración de cada algoritmo:
 
1. Random Forest:
- n_estimators=100: 100 árboles para balance entre rendimiento y velocidad
- max_depth=10: Limita profundidad para controlar overfitting
- min_samples_split=5: Mínimo 5 muestras para dividir un nodo
- min_samples_leaf=2: Mínimo 2 muestras en hojas terminales
- Estrategia: Bagging (bootstrap + promedio) para reducir varianza
 
2. Gradient Boosting:
- n_estimators=100: 100 iteraciones de boosting
- max_depth=5: Árboles más simples (stumps mejorados)
- Mismos parámetros de regularización que Random Forest
- Estrategia: Boosting secuencial que corrige errores previos
 
3. XGBoost:
- n_estimators=50: Menos iteraciones (XGBoost es más eficiente)
- max_depth=3: Árboles muy simples para evitar overfitting
- reg_alpha=1.0, reg_lambda=2.0: Regularización L1 y L2
- gamma=0.5: Penalización por complejidad del árbol
- Estrategia: Gradient boosting optimizado con regularización avanzada
 
Diferencias clave entre algoritmos:
- Random Forest: Paralelo, robusto, menos prone a overfitting
- Gradient Boosting: Secuencial, más expressivo, puede overfittear
- XGBoost: Gradient boosting optimizado, mejor regularización, más rápido
 
Evaluación inmediata:
Calculamos métricas base para establecer benchmark antes de optimización

In [7]:
# 4. MODELOS ENSEMBLE
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, min_samples_split=5, min_samples_leaf=2),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42, max_depth=5, min_samples_split=5, min_samples_leaf=2),
    'XGBoost': XGBRegressor(n_estimators=50, random_state=42, eval_metric='rmse', max_depth=3, reg_alpha=1.0, reg_lambda=2.0, gamma=0.5)
}

# Entrenar y evaluar modelos básicos
results = {}
for name, model in models.items():
    print(f'\n=== {name} BÁSICO ===')
    model.fit(X_train_scaled, y_train_noisy)  # Usar y_train_noisy
    y_pred_test = model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test_noisy, y_pred_test)  # Usar y_test_noisy
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_noisy, y_pred_test)
    mae = mean_absolute_error(y_test_noisy, y_pred_test)
    
    results[name] = {'MSE': mse, 'RMSE': rmse, 'R²': r2, 'MAE': mae}
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")


=== Random Forest BÁSICO ===
MSE: 232561.3204
RMSE: 482.2461
R²: 0.7586
MAE: 388.5226

=== Gradient Boosting BÁSICO ===
MSE: 252435.1228
RMSE: 502.4292
R²: 0.7380
MAE: 397.0668

=== XGBoost BÁSICO ===
MSE: 265444.2876
RMSE: 515.2129
R²: 0.7245
MAE: 412.0864


### Análisis Diagnóstico Preliminar
 
Análisis de correlaciones:
- Identifica qué características tienen mayor relación lineal con la variable objetivo
- Ordenamiento descendente muestra las características más predictivas
- Interpretación: Valores |r| > 0.5 son correlaciones moderadas-fuertes
 
Validación cruzada de XGBoost:
- ¿Por qué solo XGBoost?: Es típicamente el algoritmo de mayor rendimiento
- 5-fold CV: Proporciona estimación robusta del rendimiento esperado
- Scoring='r2': Métrica de varianza explicada, fácil de interpretar
- Intervalos de confianza: ±2 desviaciones estándar (aprox. 95% confianza)
 
Propósito del diagnóstico:
- Baseline establecido: Rendimiento antes de optimización
- Identificación de características: Cuáles son más importantes
- Estimación de estabilidad: Qué tan consistente es el modelo

Uso de datos con ruido:
- Todas las evaluaciones usan y_train_noisy y y_test_noisy
- Mantiene consistencia con la estrategia de realismo de datos
- Permite comparación directa con el notebook SVR

In [8]:
print("\n=== DIAGNÓSTICO ADICIONAL ===")

# Correlaciones entre características y variable objetivo
correlations = X_train.corrwith(y_train_noisy)  # Usar y_train_noisy
print("\nCorrelaciones entre características y 'Energy Consumption' (con ruido):")
print(correlations.sort_values(ascending=False))

# Validación cruzada para XGBoost básico
cv_scores = cross_val_score(models['XGBoost'], X_train_scaled, y_train_noisy, cv=5, scoring='r2')  # Usar y_train_noisy
print(f"\nValidación cruzada (5-fold) para XGBoost - R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")


=== DIAGNÓSTICO ADICIONAL ===

Correlaciones entre características y 'Energy Consumption' (con ruido):
Square Footage               0.694921
Building Type_Industrial     0.369221
Number of Occupants          0.305407
Appliances Used              0.283204
Day of Week_Weekday          0.018533
Average Temperature         -0.014925
Day of Week_Weekend         -0.018533
Building Type_Commercial    -0.030466
Building Type_Residential   -0.330685
dtype: float64

Validación cruzada (5-fold) para XGBoost - R²: 0.7505 (+/- 0.0595)


### Comparación Exhaustiva de Métodos de Optimización de Hiperparámetros
 
Estrategia experimental:
Se implementan tres enfoques diferentes para optimizar XGBoost y determinar cuál es más efectivo para este problema específico.
 
1. GridSearchCV - Búsqueda Exhaustiva:
- Ventajas: Garantiza encontrar el óptimo dentro del espacio definido
- Limitaciones: Computacionalmente costoso, crece exponencialmente
- Espacio reducido: 2×2×2×2×2×2×2 = 128 combinaciones
- Uso típico: Espacios pequeños, cuando hay tiempo suficiente
 
2. RandomizedSearchCV - Búsqueda Aleatoria:
- Ventajas: Más eficiente, explora mejor espacios de alta dimensionalidad
- n_iter=50: Evalúa 50 combinaciones aleatorias
- Espacio completo: Usa todas las opciones del param_grid
- Uso típico: Exploración inicial, espacios grandes
 
3. BayesSearchCV - Optimización Bayesiana:
- Ventajas: Más inteligente, usa información de evaluaciones previas
- Espacios continuos: Real() permite explorar valores intermedios
- Prior='log-uniform': Para learning_rate, explora escalas logarítmicas
- Uso típico: Problemas complejos, presupuesto limitado de evaluaciones
 
Definición de espacios de búsqueda:

Parámetros clave de XGBoost:
- n_estimators: Número de árboles (más = mejor ajuste, más overfitting)
- max_depth: Profundidad máxima (más = más complejo, más overfitting)
- learning_rate: Tasa de aprendizaje (menor = más conservador, necesita más árboles)
- subsample: Fracción de muestras por árbol (< 1.0 = regularización)
- colsample_bytree: Fracción de características por árbol (regularización)
- reg_alpha: Regularización L1 (sparsity, selección de características)
- reg_lambda: Regularización L2 (suavidad, previene overfitting)
- gamma: Penalización mínima para división (más = más conservador)
 
Estrategia de evaluación:
- 5-fold CV: Balance entre confiabilidad y costo computacional
- neg_mean_squared_error: Optimiza directamente la métrica objetivo
- n_jobs=-1: Paralelización para acelerar búsqueda
 
Selección automática del mejor método:
- Compara MSE en conjunto de prueba
- Selecciona automáticamente el modelo con menor error
- Proporciona trazabilidad completa del proceso de selección
 

In [9]:
# 5. OPTIMIZACIÓN CON BAYESIAN SEARCH PARA XGBoost (ejemplo, puedes extender a otros)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np

# Definir espacio de hiperparámetros común
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 8],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0.0, 0.5, 1.0],
    'reg_lambda': [0.0, 1.0, 2.0],
    'gamma': [0.0, 0.3, 0.5]
}

# 1. GridSearchCV
print("\n--- GridSearchCV ---")
grid_search = GridSearchCV(
    estimator=XGBRegressor(random_state=42, eval_metric='rmse'),
    param_grid={
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'learning_rate': [0.1, 0.3],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha': [0.0, 1.0],
        'reg_lambda': [1.0, 2.0]
    },
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train_scaled, y_train_noisy)  # Usar y_train_noisy
grid_best = grid_search.best_estimator_
grid_score = -grid_search.best_score_
print(f"Mejores parámetros (GridSearchCV): {grid_search.best_params_}")
print(f"Mejor MSE CV (GridSearchCV): {grid_score:.4f}")

# 2. RandomizedSearchCV
print("\n--- RandomizedSearchCV ---")
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42, eval_metric='rmse'),
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)
random_search.fit(X_train_scaled, y_train_noisy)  # Usar y_train_noisy
random_best = random_search.best_estimator_
random_score = -random_search.best_score_
print(f"Mejores parámetros (RandomizedSearchCV): {random_search.best_params_}")
print(f"Mejor MSE CV (RandomizedSearchCV): {random_score:.4f}")

# 3. BayesSearchCV
print("\n--- BayesSearchCV ---")
bayes_search = BayesSearchCV(
    estimator=XGBRegressor(random_state=42, eval_metric='rmse'),
    search_spaces={
        'n_estimators': Integer(50, 200),
        'max_depth': Integer(3, 8),
        'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
        'subsample': Real(0.6, 1.0),
        'colsample_bytree': Real(0.6, 1.0),
        'reg_alpha': Real(0.0, 1.0),
        'reg_lambda': Real(0.0, 2.0),
        'gamma': Real(0.0, 0.5)
    },
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)
bayes_search.fit(X_train_scaled, y_train_noisy)  # Usar y_train_noisy y todo el conjunto de entrenamiento
bayes_best = bayes_search.best_estimator_
bayes_score = -bayes_search.best_score_
print(f"Mejores parámetros (BayesSearchCV): {bayes_search.best_params_}")
print(f"Mejor MSE CV (BayesSearchCV): {bayes_score:.4f}")

# No reentrenar con early stopping debido a incompatibilidad; usar bayes_best directamente

# Comparar resultados
print("\n=== COMPARACIÓN DE MÉTODOS DE OPTIMIZACIÓN ===")
opt_results = {}
for name, model in [('GridSearchCV', grid_best), ('RandomizedSearchCV', random_best), ('BayesSearchCV', bayes_best)]:
    y_pred_test = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test_noisy, y_pred_test)  # Usar y_test_noisy
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_noisy, y_pred_test)
    mae = mean_absolute_error(y_test_noisy, y_pred_test)
    opt_results[name] = {'MSE': mse, 'RMSE': rmse, 'R²': r2, 'MAE': mae}
    print(f"\n{name} (Prueba):")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")

# Seleccionar el mejor modelo
best_method = min(opt_results, key=lambda x: opt_results[x]['MSE'])
ensemble_optimized = {'GridSearchCV': grid_best, 'RandomizedSearchCV': random_best, 'BayesSearchCV': bayes_best}[best_method]
print(f"\nMejor método: {best_method} (MSE: {opt_results[best_method]['MSE']:.4f})")


--- GridSearchCV ---
Fitting 5 folds for each of 128 candidates, totalling 640 fits
Mejores parámetros (GridSearchCV): {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'subsample': 0.8}
Mejor MSE CV (GridSearchCV): 243796.0891

--- RandomizedSearchCV ---
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Mejores parámetros (RandomizedSearchCV): {'subsample': 0.6, 'reg_lambda': 1.0, 'reg_alpha': 0.0, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.6}
Mejor MSE CV (RandomizedSearchCV): 249594.2863

--- BayesSearchCV ---
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totallin

### Evaluación Completa del Modelo Ensemble Optimizado
 
Análisis del mejor modelo seleccionado:
El modelo ensemble_optimized corresponde al algoritmo que obtuvo el menor MSE entre los tres métodos de optimización probados.
 
Evaluación en ambos conjuntos:
 
Conjunto de entrenamiento:
- Propósito: Verificar capacidad de ajuste del modelo
- Interpretación: Qué tan bien "recuerda" los datos de entrenamiento
- Señal de alerta: Si es demasiado perfecto (R² ≈ 1), posible overfitting
 
Conjunto de prueba:
- Más importante: Mide capacidad real de generalización
- Interpretación: Rendimiento esperado en datos nuevos
- Métrica crítica: Esta es la que realmente importa para producción
 
Detección de overfitting:
- Gap train-test pequeño: Modelo bien balanceado
- Gap train-test grande: Posible overfitting
- Test mejor que train: Posible underfitting (raro) o casualidad estadística
 
Validación cruzada adicional:
- ¿Por qué otra CV?: Confirmación independiente del rendimiento
- Datos originales: Usa y_train sin ruido para comparación
- Intervalos de confianza: Proporciona rango de rendimiento esperado
 
Métricas complementarias:
- MSE: Penaliza errores grandes, útil para optimización
- RMSE: Mismas unidades que la variable objetivo, interpretable
- R²: Porcentaje de varianza explicada, fácil de comunicar
- MAE: Robusto a outliers, error promedio típico


In [10]:
# 6. MODELO OPTIMIZADO (usamos XGBoost optimizado como ejemplo principal)
y_pred_train_opt = ensemble_optimized.predict(X_train_scaled)
y_pred_test_opt = ensemble_optimized.predict(X_test_scaled)

# Métricas para entrenamiento
mse_train_opt = mean_squared_error(y_train_noisy, y_pred_train_opt)  # Usar y_train_noisy
rmse_train_opt = np.sqrt(mse_train_opt)
r2_train_opt = r2_score(y_train_noisy, y_pred_train_opt)
mae_train_opt = mean_absolute_error(y_train_noisy, y_pred_train_opt)

# Métricas para prueba
mse_test_opt = mean_squared_error(y_test_noisy, y_pred_test_opt)  # Usar y_test_noisy
rmse_test_opt = np.sqrt(mse_test_opt)
r2_test_opt = r2_score(y_test_noisy, y_pred_test_opt)
mae_test_opt = mean_absolute_error(y_test_noisy, y_pred_test_opt)

print(f'\n=== MODELO OPTIMIZADO (ENTRENAMIENTO) ===')
print(f'MSE: {mse_train_opt:.4f}')
print(f'RMSE: {rmse_train_opt:.4f}')
print(f'R²: {r2_train_opt:.4f}')
print(f'MAE: {mae_train_opt:.4f}')

print(f'\n=== MODELO OPTIMIZADO (PRUEBA) ===')
print(f'MSE: {mse_test_opt:.4f}')
print(f'RMSE: {rmse_test_opt:.4f}')
print(f'R²: {r2_test_opt:.4f}')
print(f'MAE: {mae_test_opt:.4f}')

# Validación cruzada
cv_scores = cross_val_score(ensemble_optimized, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'\nCV RMSE: {np.sqrt(-cv_scores.mean()):.4f} (+/- {np.sqrt(cv_scores.std() * 2):.4f})')



=== MODELO OPTIMIZADO (ENTRENAMIENTO) ===
MSE: 152717.0443
RMSE: 390.7903
R²: 0.8594
MAE: 312.1552

=== MODELO OPTIMIZADO (PRUEBA) ===
MSE: 238829.9856
RMSE: 488.7023
R²: 0.7521
MAE: 393.0759

CV RMSE: 80.5791 (+/- 33.7801)


### Persistencia de Predicciones para Análisis Detallado
 
Estructura de archivos generados:
 
Columnas incluidas:
- Valores Reales: Variable objetivo con ruido (y_train_noisy, y_test_noisy)
- Predicciones: Salida del modelo ensemble optimizado
- Diferencia: Error residual (real - predicho)
 
Utilidades de estos archivos:
 
Análisis post-hoc:
- Identificación de outliers: Casos con errores muy grandes
- Análisis de patrones: ¿En qué rangos el modelo falla más?
- Distribución de errores: ¿Son normalmente distribuidos?
 
Comparación entre modelos:
- Consistencia: Comparar con predicciones de SVR
- Benchmarking: Usar como baseline para futuros modelos
- Ensemble de modelos: Combinar predicciones de diferentes enfoques
 
Validación de negocio:
- Casos críticos: Identificar predicciones problemáticas
- Intervalos de confianza: Estimar incertidumbre de predicciones
- Análisis de sensibilidad: ¿Qué características afectan más las diferencias?

Uso de datos con ruido:
- Mantiene consistencia con la estrategia experimental
- Permite análisis comparativo directo con otros notebooks
- Refleja incertidumbre realista en los datos

In [11]:
# 7. GUARDAR RESULTADOS EN CSV
train_results = pd.DataFrame({
    'Valores Reales': y_train_noisy,  # Usar y_train_noisy
    'Predicciones': y_pred_train_opt,
    'Diferencia': y_train_noisy - y_pred_train_opt
})
train_results.to_csv('../data/results/ensemble_predictions_train.csv', index=False)
print("\nPredicciones de entrenamiento guardadas en '../data/results/ensemble_predictions_train.csv'")

test_results = pd.DataFrame({
    'Valores Reales': y_test_noisy,  # Usar y_test_noisy
    'Predicciones': y_pred_test_opt,
    'Diferencia': y_test_noisy - y_pred_test_opt
})
test_results.to_csv('../data/results/ensemble_predictions_test.csv', index=False)
print("Predicciones de prueba guardadas en '../data/results/ensemble_predictions_test.csv'")


Predicciones de entrenamiento guardadas en '../data/results/ensemble_predictions_train.csv'
Predicciones de prueba guardadas en '../data/results/ensemble_predictions_test.csv'


### Panel Completo de Visualizaciones Diagnósticas
 
Estrategia de visualización en dos paneles:
 
PANEL 1 - Conjunto de Prueba (2×2):
El más crítico porque evalúa la capacidad de generalización real
 
Gráfico Superior Izquierda - Predicciones vs Valores Reales:
- Línea diagonal roja: Referencia de predicción perfecta (y = x)
- Dispersión de puntos: Indica variabilidad de errores
- Concentración en diagonal: Señal de buenas predicciones
- Patrones sistemáticos: Podrían indicar bias del modelo
 
Gráfico Superior Derecha - Análisis de Residuos:
- Línea horizontal en y=0: Referencia de error cero
- Distribución aleatoria: Indicador de modelo bien especificado
- Patrones o tendencias: Señales de problemas (heterocedasticidad, no linealidad)
- Outliers extremos: Casos que el modelo predice mal
 
Gráficos Inferiores - Comparación de Modelos:
- MSE (Izquierda): Comparación de errores absolutos
- R² (Derecha): Comparación de capacidad explicativa
- 6 modelos totales: 3 básicos + 3 métodos de optimización
- Rotación de etiquetas: Evita solapamiento de nombres
 
PANEL 2 - Conjunto de Entrenamiento (1×2):
 Complementa el análisis para detectar overfitting
 
Interpretación conjunta Train vs Test:
- Similaridad: Modelo bien balanceado
- Train perfecto, Test imperfecto: Overfitting
- Ambos imperfectos pero similares: Underfitting controlado
 
Detalles técnicos:
- alpha=0.6: Transparencia para ver densidad de puntos
- dpi=300: Alta resolución para publicación
- bbox_inches='tight': Elimina espacios innecesarios
- plt.close(): Libera memoria después de guardar

In [12]:
# 8. VISUALIZACIONES
output_dir = "../data/figures/"

# Visualizaciones para el conjunto de prueba
fig_test, axes_test = plt.subplots(2, 2, figsize=(15, 12))

axes_test[0,0].scatter(y_test_noisy, y_pred_test_opt, alpha=0.6)  # Usar y_test_noisy
axes_test[0,0].plot([y_test_noisy.min(), y_test_noisy.max()], [y_test_noisy.min(), y_test_noisy.max()], 'r--', lw=2)
axes_test[0,0].set_xlabel('Valores Reales')
axes_test[0,0].set_ylabel('Predicciones')
axes_test[0,0].set_title(f'Ensemble Optimizado (Prueba) - R² = {r2_test_opt:.4f}')
axes_test[0,0].grid(True, alpha=0.3)

residuos_test = y_test_noisy - y_pred_test_opt  # Usar y_test_noisy
axes_test[0,1].scatter(y_pred_test_opt, residuos_test, alpha=0.6)
axes_test[0,1].axhline(y=0, color='r', linestyle='--')
axes_test[0,1].set_xlabel('Predicciones')
axes_test[0,1].set_ylabel('Residuos')
axes_test[0,1].set_title('Gráfico de Residuos (Prueba)')
axes_test[0,1].grid(True, alpha=0.3)

model_names = list(results.keys()) + ['GridSearchCV', 'RandomizedSearchCV', 'BayesSearchCV']
mse_values = [results[m]['MSE'] for m in results] + [opt_results[m]['MSE'] for m in ['GridSearchCV', 'RandomizedSearchCV', 'BayesSearchCV']]
r2_values = [results[m]['R²'] for m in results] + [opt_results[m]['R²'] for m in ['GridSearchCV', 'RandomizedSearchCV', 'BayesSearchCV']]

x_pos = np.arange(len(model_names))
axes_test[1,0].bar(x_pos, mse_values, alpha=0.7)
axes_test[1,0].set_xlabel('Modelo')
axes_test[1,0].set_ylabel('MSE')
axes_test[1,0].set_title('Comparación MSE por Modelo')
axes_test[1,0].set_xticks(x_pos)
axes_test[1,0].set_xticklabels(model_names, rotation=45)

axes_test[1,1].bar(x_pos, r2_values, alpha=0.7, color='green')
axes_test[1,1].set_xlabel('Modelo')
axes_test[1,1].set_ylabel('R²')
axes_test[1,1].set_title('Comparación R² por Modelo')
axes_test[1,1].set_xticks(x_pos)
axes_test[1,1].set_xticklabels(model_names, rotation=45)

plt.tight_layout()
fig_test.savefig(f"{output_dir}ensemble_comparative_analysis_test.png", dpi=300, bbox_inches='tight')
plt.close(fig_test)
print(f"📊 Gráfico de prueba guardado en: {output_dir}ensemble_comparative_analysis_test.png")

# Visualizaciones para el conjunto de entrenamiento
fig_train, axes_train = plt.subplots(1, 2, figsize=(15, 6))

axes_train[0].scatter(y_train, y_pred_train_opt, alpha=0.6)
axes_train[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes_train[0].set_xlabel('Valores Reales')
axes_train[0].set_ylabel('Predicciones')
axes_train[0].set_title(f'Ensemble Optimizado (Entrenamiento) - R² = {r2_train_opt:.4f}')
axes_train[0].grid(True, alpha=0.3)

residuos_train = y_train - y_pred_train_opt
axes_train[1].scatter(y_pred_train_opt, residuos_train, alpha=0.6)
axes_train[1].axhline(y=0, color='r', linestyle='--')
axes_train[1].set_xlabel('Predicciones')
axes_train[1].set_ylabel('Residuos')
axes_train[1].set_title('Gráfico de Residuos (Entrenamiento)')
axes_train[1].grid(True, alpha=0.3)

plt.tight_layout()
fig_train.savefig(f"{output_dir}ensemble_comparative_analysis_train.png", dpi=300, bbox_inches='tight')
plt.close(fig_train)
print(f"📊 Gráfico de entrenamiento guardado en: {output_dir}ensemble_comparative_analysis_train.png")

📊 Gráfico de prueba guardado en: ../data/figures/ensemble_comparative_analysis_test.png
📊 Gráfico de entrenamiento guardado en: ../data/figures/ensemble_comparative_analysis_train.png


## Resumen de Resultados de Modelos Ensemble
La siguiente tabla presenta un resumen comparativo del desempeño de diferentes modelos de machine learning utilizados para predecir el consumo energético (Energy Consumption) en un dataset sintético. Los modelos evaluados incluyen Random Forest, Gradient Boosting, XGBoost (versión básica), y versiones optimizadas de XGBoost mediante GridSearchCV, RandomizedSearchCV, y BayesSearchCV, junto con los resultados del modelo XGBoost optimizado en los conjuntos de entrenamiento y prueba.
 
## Descripción de las Métricas
 
- Modelo: Nombre del modelo o método de optimización evaluado.
- MSE (Error Cuadrático Medio): Promedio de los errores al cuadrado entre los valores reales y predichos, mide la magnitud de los errores.
- RMSE (Raíz del Error Cuadrático Medio): Raíz cuadrada del MSE, proporciona una medida interpretable en la misma unidad que la variable objetivo.
- R² (Coeficiente de Determinación): Indica la proporción de la varianza en la variable objetivo explicada por el modelo (valores cercanos a 1 indican mejor ajuste, ~0.6–0.9 es realista con datos ruidosos).
- MAE (Error Absoluto Medio): Promedio de los errores absolutos, mide la magnitud promedio de los errores sin considerar su dirección.

## Contexto
Los datos incluyen ruido añadido (y_train_noisy, y_test_noisy) para simular un escenario realista, ya que el dataset original es sintético con una relación lineal perfecta. Las métricas reflejan el desempeño en el conjunto de prueba (excepto para "XGBoost Optimizado (Entrenamiento)"), con valores esperados de R² ~0.6–0.9 y RMSE ~400–600, dependiendo del nivel de ruido.

## Uso
Esta tabla permite comparar el desempeño de los modelos básicos y optimizados, identificando el mejor método (menor MSE en el conjunto de prueba). Los resultados se guardan en '../data/results/ensemble_resumen_resultados.csv' para análisis posterior.

In [13]:
# 9. TABLA RESUMEN DE RESULTADOS
print("\n=== RESUMEN DE RESULTADOS ===")
resumen = pd.DataFrame({
    'Modelo': ['Random Forest', 'Gradient Boosting', 'XGBoost', 'GridSearchCV', 'RandomizedSearchCV', 'BayesSearchCV', 'XGBoost Optimizado (Entrenamiento)', 'XGBoost Optimizado (Prueba)'],
    'MSE': [results['Random Forest']['MSE'], results['Gradient Boosting']['MSE'], results['XGBoost']['MSE'], 
            opt_results['GridSearchCV']['MSE'], opt_results['RandomizedSearchCV']['MSE'], opt_results['BayesSearchCV']['MSE'], 
            mse_train_opt, mse_test_opt],
    'RMSE': [results['Random Forest']['RMSE'], results['Gradient Boosting']['RMSE'], results['XGBoost']['RMSE'], 
             opt_results['GridSearchCV']['RMSE'], opt_results['RandomizedSearchCV']['RMSE'], opt_results['BayesSearchCV']['RMSE'], 
             rmse_train_opt, rmse_test_opt],
    'R²': [results['Random Forest']['R²'], results['Gradient Boosting']['R²'], results['XGBoost']['R²'], 
           opt_results['GridSearchCV']['R²'], opt_results['RandomizedSearchCV']['R²'], opt_results['BayesSearchCV']['R²'], 
           r2_train_opt, r2_test_opt],
    'MAE': [results['Random Forest']['MAE'], results['Gradient Boosting']['MAE'], results['XGBoost']['MAE'], 
            opt_results['GridSearchCV']['MAE'], opt_results['RandomizedSearchCV']['MAE'], opt_results['BayesSearchCV']['MAE'], 
            mae_train_opt, mae_test_opt]
})

print(resumen.round(4))

# Guardar la tabla resumen como CSV
resumen.to_csv('../data/results/ensemble_resumen_resultados.csv', index=False)
print("Resumen de resultados guardado en '../data/results/ensemble_resumen_resultados.csv'")


=== RESUMEN DE RESULTADOS ===
                               Modelo          MSE      RMSE      R²       MAE
0                       Random Forest  232561.3204  482.2461  0.7586  388.5226
1                   Gradient Boosting  252435.1228  502.4292  0.7380  397.0668
2                             XGBoost  265444.2876  515.2129  0.7245  412.0864
3                        GridSearchCV  238829.9856  488.7023  0.7521  393.0759
4                  RandomizedSearchCV  242187.9053  492.1259  0.7486  390.9323
5                       BayesSearchCV  256503.1255  506.4614  0.7338  404.6343
6  XGBoost Optimizado (Entrenamiento)  152717.0443  390.7903  0.8594  312.1552
7         XGBoost Optimizado (Prueba)  238829.9856  488.7023  0.7521  393.0759
Resumen de resultados guardado en '../data/results/ensemble_resumen_resultados.csv'


In [14]:
# 10. GUARDAR EL MODELO
joblib.dump(ensemble_optimized, '../data/results/ensemble_model.pkl')
joblib.dump(scaler, '../data/results/ensemble_scaler.pkl')
print("\nModelo y scaler guardados como 'ensemble_model.pkl' y 'ensemble_scaler.pkl'")



Modelo y scaler guardados como 'ensemble_model.pkl' y 'ensemble_scaler.pkl'


In [15]:
# 11. FUNCIÓN PARA NUEVAS PREDICCIONES
def predecir_nuevos_datos(nuevos_datos, modelo=ensemble_optimized, escalador=scaler):
    """
    Función para hacer predicciones en nuevos datos
    
    Parameters:
    nuevos_datos: array-like o DataFrame, datos a predecir
    modelo: modelo ensemble entrenado
    escalador: StandardScaler ajustado
    
    Returns:
    predicciones: array con las predicciones
    """
    if isinstance(nuevos_datos, pd.DataFrame):
        nuevos_datos = nuevos_datos.values
    datos_escalados = escalador.transform(nuevos_datos)
    predicciones = modelo.predict(datos_escalados)
    return predicciones

print("\n¡Modelos ensemble implementados exitosamente!")


¡Modelos ensemble implementados exitosamente!
